In [4]:
from arhender_aiida_workchains.workchains import DiffuseImplicitHybridWorkChain
from aiida.orm import Str, Dict, load_code
from aiida_aimall.workchains import QMToAIMWorkchain
from aiida.engine import submit
from aiida.orm import load_node
from aiida import load_profile

load_profile()

builder = DiffuseImplicitHybridWorkChain.get_builder()

with open('basis_file.txt','r') as bf:
    diffuse_basis_set_data = bf.readlines()

df = ''
for line in diffuse_basis_set_data:
    df += line
df = Str(df)


g16_base_params = Dict({'charge': -1,
    'basis_set': '6-31g(d)',
    'functional': 'hf',
    'multiplicity': 1,
    'input_parameters': {},
    'link0_parameters': {'%chk': 'aiida.chk',
    '%mem': '1600MB',
    '%nprocshared': 4},
    'route_parameters': {'opt': None, 'freq': None}})


with open('structure.txt','r') as sf:
    structure = sf.readlines()
structr=''
for line in structure:
    structr += line
structr = Str(structr)

molecule_name = Str('PbiL2mL1_Conformers_13')
solvent=  Str('water')
builder.input_structure = structr
builder.solvent = solvent
builder.molecule_name = molecule_name
builder.diffuse_basis_set_data = df
builder.g16_code = load_code('gaussian@cedar')
builder.original_parsed_data = Dict({'freeenergy':123})
builder.g16_base_params = g16_base_params
submit(builder)

The aiida extension is already loaded. To reload it, use:
  %reload_ext aiida


In [2]:
%load_ext aiida
%aiida
from aiida.orm import load_code, Str, SinglefileData, List
from aiida_aimall.data import AimqbParameters
from aiida.engine import submit
input = SinglefileData('/Users/chemlab/Documents/Coding/LearningAiidaShell/test.inp')
orca_code = load_code('orca@cedar')
builder = QMToAIMWorkchain.get_builder()
builder.shell_code = orca_code
builder.shell_metadata = Dict({
        'options': {
            'withmpi': False,
            'prepend_text': 'module load StdEnv/2020; module load gcc/10.3.0 module load openmpi/4.1.1',
            'resources': {
                'num_machines': 1,
                'num_mpiprocs_per_machine': 4,
            },
            'max_memory_kb': int(3200 * 1.25) * 1024,
            'max_wallclock_seconds': 3600
        }
    })
builder.shell_retrieved = List([input.filename.replace('inp','wfx'),input.filename.replace('inp','opt'),])
builder.shell_input_file = input
builder.shell_cmdline = Str("{file}")
builder.aim_code = load_code('aimall@localhost')
builder.aim_params = AimqbParameters(parameter_dict={"naat": 2, "nproc": 2, "atlaprhocps": True})
submit(builder)

The aiida extension is already loaded. To reload it, use:
  %reload_ext aiida


<WorkChainNode: uuid: 279a7a60-6ea4-4bd1-a7fe-5ffe6fd9803d (pk: 114016) (aiida_aimall.workchains.QMToAIMWorkchain)>